In [90]:
#STILL WIP

# # Read from:
# Dune - OP Deployed by deployer address type 
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [91]:
import pandas as pd
import numpy as np
import datetime

In [92]:
tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
distrib_df = pd.read_csv('csv_outputs/dune_op_distribution_type.csv')
program_df = pd.read_csv('inputs/op_incentive_program_info.csv')
app_df = pd.read_csv('csv_outputs/dune_usage_by_app.csv')

In [93]:
#Filter TVL DF
tvl = tvl[tvl['include_in_summary'] == 1]
tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')
# display(tvl)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_59737/4163294631.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')


In [94]:
op_token_columns = ['op_claimed','op_deployed', 'op_from_other_projects','op_to_other_projects','op_to_project']

In [95]:
#Set up Distributions for Mapping
distrib_df['program_map'] = np.where(
                                distrib_df['to_type'].isin(tvl['top_level_name'])
                                ,distrib_df['to_type']
                                ,''
                                )
group_cols = ['from_name','to_label','to_type','program_map'] + op_token_columns

sum_distrib_df = distrib_df[group_cols].groupby(['from_name','program_map']).sum()
sum_distrib_df.reset_index(inplace=True)
#Joins should maybe just be the program map OR from name, since Velo operated bribes for a while
sum_distrib_df['join_key'] = np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])
                            # sum_distrib_df['from_name'] + ' - ' \
                            #     + np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])

# display(sum_distrib_df[sum_distrib_df['join_key'].str.contains('elodr')])

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_59737/15182629.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_distrib_df = distrib_df[group_cols].groupby(['from_name','program_map']).sum()


In [96]:
#lowercase joinkeys
tvl['join_key'] = tvl['join_key'].str.lower()
sum_distrib_df['join_key'] = sum_distrib_df['join_key'].str.lower()
#
df = sum_distrib_df.merge(tvl, on='join_key',how='outer')
# display(df[df['join_key'].str.contains('velodr')])

In [97]:
# Overrides as needed
def replace_program_names(df, overrides):
    for program, program_override in overrides.items():
        df.loc[df['join_key'] == program, 'from_name'] = program_override
    return df

#Overrides if needed
overrides = {
    'old name'  :   'new name',
}

# Replace program names with overrides
df = replace_program_names(df, overrides)

In [98]:
#Create the aggregate app name field
df['agg_app_name'] = df['app_name'].combine_first(df['from_name'])
df = df.fillna(0) #Fill NA with 0
# display(df)

In [99]:
# Now union back again

data_cols = ['agg_app_name','top_level_name', 'program_name',\
               'num_op_override', 'period','op_source', 'start_date','end_date',
               'cumul_net_dollar_flow_at_program_end',
                'cumul_net_dollar_flow',
                'cumul_last_price_net_dollar_flow_at_program_end',
                'cumul_last_price_net_dollar_flow'
                ]
select_cols = data_cols + op_token_columns

group_cols = select_cols[:8] #group by 1 to 8
print(group_cols)

# display(df[select_cols])

sum_distrib_df = df[select_cols].groupby(group_cols).sum()
sum_distrib_df.reset_index(inplace=True)

['agg_app_name', 'top_level_name', 'program_name', 'num_op_override', 'period', 'op_source', 'start_date', 'end_date']


In [100]:
#Get the rank by start_date of each program

sum_distrib_df = sum_distrib_df.reset_index().rename(columns={'index': 'row_num'})

# replace 0s with '9999-12-31'
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == 0, '9999-12-31', sum_distrib_df['start_date'])
# create a new column 'program_rank' based on the 'start_date' column
sum_distrib_df = sum_distrib_df.sort_values(['agg_app_name', 'start_date', 'row_num'])
sum_distrib_df['program_rank'] = sum_distrib_df.groupby('agg_app_name').cumcount() + 1

sum_distrib_df = sum_distrib_df.sort_values(by=['agg_app_name','program_rank'],ascending=[True,True])
# subtract all overridden values fromthe amount I have deployed

# create a new column 'cumulative_num_op_override' that contains the cumulative sum of 'num_op_override' for each agg_app_name group
sum_distrib_df['cumulative_num_op_override'] = sum_distrib_df.groupby('agg_app_name')['num_op_override'].cumsum()


# create a new column 'op_deployed_net_override' that subtracts 'cumulative_num_op_override' from 'op_deployed'
sum_distrib_df['op_deployed_net_override'] = sum_distrib_df['op_deployed'] - sum_distrib_df.groupby('agg_app_name')['cumulative_num_op_override'].shift(1).fillna(0)
# drop the 'cumulative_num_op_override' column
# sum_distrib_df.drop('cumulative_num_op_override', axis=1, inplace=True)

# replace '9999-12-31' with 0s
sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == '9999-12-31', 0, sum_distrib_df['start_date'])

#Drop Row Num
sum_distrib_df.drop('row_num', axis=1, inplace=True)
# display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('rrakis')])

In [101]:
# Now do the algorithmic overrides - where we want to redistirbute deployed OP across specific programs (i.e. Uniswap LM w/ Partners)
# # replace 0s in 'num_op_override' with the corresponding value in 'op_deployed_net_override'
sum_distrib_df['og_op_deployed'] = sum_distrib_df['op_deployed'] 
# Override # OP Deployed 
sum_distrib_df['op_deployed'] = np.where((sum_distrib_df['num_op_override'] == 0)
                                         , sum_distrib_df['op_deployed_net_override']
                                         , sum_distrib_df['num_op_override']
                                         )

#Hardcode for Aave - Liquidity Mining since claims came straight from the FND wallet. This should be a one-time edge case
sum_distrib_df['op_deployed'] = np.where(sum_distrib_df['top_level_name'] == 'Aave - Liquidity Mining'
                                         , 5_000_000
                                         , sum_distrib_df['op_deployed']
                                         )

In [102]:

# #Select all except the last 4 rows
sum_distrib_df = sum_distrib_df.iloc[:, :-4]


In [103]:
# latest tvl metrics
sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end_ended'] = np.where(sum_distrib_df['period']=='Post-Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end']\
            ,np.nan\
)

sum_distrib_df['cumul_last_price_net_dollar_flow_ended'] = np.where(sum_distrib_df['period']=='Post-Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow']\
            ,np.nan\
)
sum_distrib_df['net_flows_retention'] = np.where(sum_distrib_df['period']=='Post-Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow_ended'] / sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end_ended']\
            ,np.nan\
)
# If < 0 then make retention 0
sum_distrib_df['net_flows_retention'] = np.where(sum_distrib_df['cumul_last_price_net_dollar_flow']<0\
            ,0\
            ,sum_distrib_df['net_flows_retention']\
)

# Live Programs

sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end_live'] = np.where(sum_distrib_df['period']=='During Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end']\
            ,np.nan\
)

sum_distrib_df['cumul_last_price_net_dollar_flow_live'] = np.where(sum_distrib_df['period']=='During Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow']\
            ,np.nan\
)

In [104]:
# Get App Name Mappings from Notion
name_mappings = program_df[['App Name', 'App Name Map Override']].drop_duplicates()
name_mappings = name_mappings[~name_mappings['App Name Map Override'].isna()]
name_mappings = name_mappings.rename(columns={'App Name':'agg_app_name'})
sum_distrib_df = sum_distrib_df.merge(name_mappings, on='agg_app_name',how='left')
sum_distrib_df['App Name Map'] = (sum_distrib_df['App Name Map Override'].combine_first(sum_distrib_df['agg_app_name'])).str.lower()

sum_distrib_df['last_updated'] = pd.to_datetime(datetime.datetime.now())
sum_distrib_df.to_csv('csv_outputs/incentives_stats_summary.csv')
display(sum_distrib_df)
# display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('rrakis')])

,agg_app_name,top_level_name,program_name,num_op_override,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,...,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,cumul_last_price_net_dollar_flow_ended,net_flows_retention,cumul_last_price_net_dollar_flow_at_program_end_live,cumul_last_price_net_dollar_flow_live,App Name Map Override,App Name Map,last_updated
0,1inch,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,...,0.0,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,1inch,2023-03-13 19:09:34.985731
1,Aave,Aave - Liquidity Mining,Aave - Aave - Liquidity Mining,0.0,Post-Program,Partner Fund,2022-08-04,2022-11-04,3.450056e+08,7.849483e+07,...,0.0,0.0,3.419694e+08,7.732963e+07,0.226130,NaN,NaN,NaN,Aave,2023-03-13 19:09:34.985731
2,Aave,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,...,0.0,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,Aave,2023-03-13 19:09:34.985731
3,Aelin,Aelin,Aelin,0.0,Post-Program,Governance - Phase 0,2022-09-12,2022-09-14,-2.827652e+03,-2.778652e+03,...,19827.0,900000.0,-2.826693e+03,-2.777707e+03,0.000000,NaN,NaN,NaN,Aelin,2023-03-13 19:09:34.985731
4,Alchemix,alETH/WETH: Velodrome,Alchemix - alETH/WETH: Velodrome,0.0,During Program,Governance - Season 1,2023-01-08,0,8.205935e+06,8.205935e+06,...,0.0,0.0,NaN,NaN,NaN,8.158590e+06,8.158590e+06,NaN,Alchemix,2023-03-13 19:09:34.985731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Xtoken,Uniswap LM - Phase 2,Xtoken - Uniswap LM - Phase 2,25000.0,Post-Program,Governance - Phase 0,2023-01-16,2023-02-08,2.823585e+06,-3.557705e+04,...,0.0,0.0,3.027949e+06,7.479739e+04,0.024702,NaN,NaN,NaN,Xtoken,2023-03-13 19:09:34.985731
101,Yearn,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,...,0.0,1000000.0,NaN,NaN,NaN,NaN,NaN,NaN,Yearn,2023-03-13 19:09:34.985731
102,Zeroex (0x),0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,...,0.0,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,Zeroex (0x),2023-03-13 19:09:34.985731
103,Zipswap,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,...,0.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,Zipswap,2023-03-13 19:09:34.985731


In [105]:
# sum_distrib_df_grouped = sum_distrib_df


In [106]:
# assuming your dataframe is named `df`
program_df['App Name Map'] = (program_df['App Name Map Override'].combine_first(program_df['App Name'])).str.lower()
program_df_grouped = program_df.groupby('App Name Map').agg({
    '# OP Allocated': 'sum',
    'Source': lambda x: list(set(x))
}).reset_index()

# reset_index() is used to convert the grouped result back to a dataframe
# the lambda function for 'Source' column aggregates text entries as a list/array
display(program_df_grouped)

# Get Deployments Grouped
sum_distrib_df_grouped = sum_distrib_df.groupby('App Name Map').agg(sum).reset_index()
sum_distrib_df_grouped['net_flows_retention'] = np.where(sum_distrib_df_grouped['cumul_last_price_net_dollar_flow_ended']<0,0,\
                                                         sum_distrib_df_grouped['cumul_last_price_net_dollar_flow_ended'] / sum_distrib_df_grouped['cumul_last_price_net_dollar_flow_at_program_end_ended']\
)
sum_distrib_df_grouped.drop(['num_op_override','cumul_net_dollar_flow_at_program_end','cumul_net_dollar_flow'],axis=1,inplace=True)
display(sum_distrib_df_grouped)
sum_distrib_df_grouped.to_csv('csv_outputs/incentives_summary_by_app.csv')
print(sum_distrib_df_grouped['op_deployed'].sum())

print(sum_distrib_df_grouped.columns)

,App Name Map,# OP Allocated,Source
0,1inch,300000.0,[Governance - Phase 0]
1,Aave,5300000.0,"[Partner Fund, Governance - Phase 0]"
2,Across protocol,750000.0,[Governance - Season 2]
3,Aelin,900000.0,[Governance - Phase 0]
4,Agora,50000.0,[Governance - Season 2]
...,...,...,...
84,Wepiggy,300000.0,[Governance - Phase 0]
85,Xtoken,900000.0,[Governance - Season 1]
86,Yearn,1000000.0,[Governance - Season 2]
87,Zeroex (0x),300000.0,[Governance - Phase 0]


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_59737/118473127.py:13: FutureWarning: The operation <built-in function sum> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  sum_distrib_df_grouped = sum_distrib_df.groupby('App Name Map').agg(sum).reset_index()


,App Name Map,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,cumul_last_price_net_dollar_flow_ended,net_flows_retention,cumul_last_price_net_dollar_flow_at_program_end_live,cumul_last_price_net_dollar_flow_live
0,1inch,0.000000e+00,0.000000e+00,25517.0,300000.0,0.0,0.0,300000.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00
1,Aave,3.419694e+08,7.732963e+07,4820781.0,5000000.0,0.0,0.0,300000.0,3.419694e+08,7.732963e+07,0.226130,0.000000e+00,0.000000e+00
2,Aelin,-2.826693e+03,-2.777707e+03,880175.0,900002.0,0.0,19827.0,900000.0,-2.826693e+03,-2.777707e+03,0.000000,0.000000e+00,0.000000e+00
3,Alchemix,8.158590e+06,8.158590e+06,21722.0,25263.0,0.0,0.0,250000.0,0.000000e+00,0.000000e+00,NaN,8.158590e+06,8.158590e+06
4,Angle,-1.130590e+05,-1.130590e+05,25892.0,26373.0,0.0,0.0,250976.0,0.000000e+00,0.000000e+00,NaN,-1.130590e+05,-1.130590e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Wepiggy,1.202206e+05,1.202206e+05,299618.0,300002.0,0.0,0.0,300008.0,0.000000e+00,0.000000e+00,NaN,1.202206e+05,1.202206e+05
66,Xtoken,6.039382e+06,2.036158e+06,4109885.0,425504.0,83334.0,0.0,600008.0,5.165498e+06,1.162274e+06,0.225007,8.738845e+05,8.738845e+05
67,Yearn,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,1000000.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00
68,Zeroex (0x),0.000000e+00,0.000000e+00,452.0,452.0,0.0,0.0,300000.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00


225914546.0
Index(['App Name Map', 'cumul_last_price_net_dollar_flow_at_program_end',
       'cumul_last_price_net_dollar_flow', 'op_claimed', 'op_deployed',
       'op_from_other_projects', 'op_to_other_projects', 'op_to_project',
       'cumul_last_price_net_dollar_flow_at_program_end_ended',
       'cumul_last_price_net_dollar_flow_ended', 'net_flows_retention',
       'cumul_last_price_net_dollar_flow_at_program_end_live',
       'cumul_last_price_net_dollar_flow_live'],
      dtype='object')


In [107]:
joined_df = sum_distrib_df_grouped.merge(program_df_grouped, on='App Name Map',how='outer')
display(joined_df)

,App Name Map,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,cumul_last_price_net_dollar_flow_ended,net_flows_retention,cumul_last_price_net_dollar_flow_at_program_end_live,cumul_last_price_net_dollar_flow_live,# OP Allocated,Source
0,1inch,0.000000e+00,0.000000e+00,25517.0,300000.0,0.0,0.0,300000.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,300000.0,[Governance - Phase 0]
1,Aave,3.419694e+08,7.732963e+07,4820781.0,5000000.0,0.0,0.0,300000.0,3.419694e+08,7.732963e+07,0.22613,0.000000e+00,0.000000e+00,5300000.0,"[Partner Fund, Governance - Phase 0]"
2,Aelin,-2.826693e+03,-2.777707e+03,880175.0,900002.0,0.0,19827.0,900000.0,-2.826693e+03,-2.777707e+03,0.00000,0.000000e+00,0.000000e+00,900000.0,[Governance - Phase 0]
3,Alchemix,8.158590e+06,8.158590e+06,21722.0,25263.0,0.0,0.0,250000.0,0.000000e+00,0.000000e+00,NaN,8.158590e+06,8.158590e+06,250000.0,[Governance - Season 2]
4,Angle,-1.130590e+05,-1.130590e+05,25892.0,26373.0,0.0,0.0,250976.0,0.000000e+00,0.000000e+00,NaN,-1.130590e+05,-1.130590e+05,250000.0,[Governance - Season 2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Synapse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000.0,[Governance - Phase 0]
97,Tally ho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400000.0,[Governance - Season 2]
98,The optimistic series,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,[Governance - Season 3]
99,Uniswap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000.0,[Governance - Phase 0]


In [108]:
app_cols = ['app_name'
        ,'txs_per_day_prev','num_addr_per_day_prev','gas_fee_eth_per_day_prev'
        ,'txs_per_day','num_addr_per_day','gas_fee_eth_per_day'
        ,'txs_per_day_after','num_addr_per_day_after','gas_fee_eth_per_day_after']
app_df_sm = app_df[app_cols]
app_df_sm['app_name'] = app_df_sm['app_name'].str.lower()
app_df_sm = app_df_sm.rename(columns={'app_name':'App Name Map'})

In [109]:
joined_df = joined_df.merge(app_df_sm,on='App Name Map',how='outer')

display(joined_df)

joined_df.to_csv('csv_outputs/total_stats_summary_by_app.csv')

,App Name Map,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,cumul_last_price_net_dollar_flow_ended,...,Source,txs_per_day_prev,num_addr_per_day_prev,gas_fee_eth_per_day_prev,txs_per_day,num_addr_per_day,gas_fee_eth_per_day,txs_per_day_after,num_addr_per_day_after,gas_fee_eth_per_day_after
0,1inch,0.000000e+00,0.000000e+00,25517.0,300000.0,0.0,0.0,300000.0,0.000000e+00,0.000000e+00,...,[Governance - Phase 0],1137.500000,408.700000,1.577298,3156.000000,759.000000,4.221651,1522.689655,756.965517,1.537847
1,Aave,3.419694e+08,7.732963e+07,4820781.0,5000000.0,0.0,0.0,300000.0,3.419694e+08,7.732963e+07,...,"[Partner Fund, Governance - Phase 0]",247.733333,125.966667,0.389934,1648.462366,943.978495,4.436537,2439.862069,1466.000000,7.712612
2,Aelin,-2.826693e+03,-2.777707e+03,880175.0,900002.0,0.0,19827.0,900000.0,-2.826693e+03,-2.777707e+03,...,[Governance - Phase 0],9.700000,6.100000,0.068525,15.666667,9.000000,0.098215,4.965517,3.620690,0.031710
3,Alchemix,8.158590e+06,8.158590e+06,21722.0,25263.0,0.0,0.0,250000.0,0.000000e+00,0.000000e+00,...,[Governance - Season 2],6.833333,3.500000,0.050856,15.000000,8.609375,0.194009,NaN,NaN,NaN
4,Angle,-1.130590e+05,-1.130590e+05,25892.0,26373.0,0.0,0.0,250976.0,0.000000e+00,0.000000e+00,...,[Governance - Season 2],14.100000,11.166667,0.261447,43.333333,29.428571,0.948608,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,dHedge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,160.200000,60.966667,0.422789,323.231707,161.695122,1.084440,NaN,NaN,NaN
120,WardenSwap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,27.733333,18.766667,0.003312,34.333333,20.666667,0.006812,NaN,NaN,NaN
121,Rocket Pool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,QiDao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,120.233333,50.333333,0.472826,1045.668342,343.190955,3.953424,NaN,NaN,NaN
